In [10]:
import wrds

In [11]:
with wrds.Connection(wrds_username="mayurankv") as db:
	db.list_libraries().sort()
	db.describe_table(library="crsp", table="stocknames")

Loading library list...


Done
Approximately 80790 rows in crsp.stocknames.


In [ ]:
query = """
SELECT COUNT(*)
FROM crsp_a_indexes.dsp500list_v2
WHERE NOT (mbrenddt < '2023-12-29')
"""

result = db.raw_sql(query)
print(result)

   count
0    503


In [ ]:
with wrds.Connection(wrds_username="mayurankv") as db:
	query = """
	SELECT
		date,
		secid,
		symbol,
		exdate,
		strike_price,
		cp_flag,
		best_bid,
		best_offer,
		impl_volatility,
		delta,
		gamma,
		vega,
		theta
	FROM
		optionm.optionmetrics
	WHERE
		symbol = 'SPX'
		AND date BETWEEN '2023-01-01' AND '2023-12-31'
	ORDER BY
		date, exdate, strike_price;
	"""
	# spx_options = db.raw_sql(query)
	# print(spx_options)

Loading library list...
Done


In [56]:
with wrds.Connection(wrds_username="mayurankv") as db:
	squery = """
	SELECT *
	FROM optionm_all.option_price
	WHERE underlying_id = 'SPX'
	AND date BETWEEN '2023-01-01' AND '2023-12-31'
	"""

	# Execute the query and fetch the data
	option_prices = db.raw_sql(squery)

	# Display the first few rows of the data
	print(option_prices.head())

Loading library list...
Done


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "optionm_all.option_price" does not exist
LINE 3:  FROM optionm_all.option_price
              ^

[SQL: 
	SELECT *
	FROM optionm_all.option_price
	WHERE underlying_id = 'SPX'
	AND date BETWEEN '2023-01-01' AND '2023-12-31'
	]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [57]:
with wrds.Connection(wrds_username="mayurankv") as db:
	query = """
	SELECT *
	FROM optionm_all.opprcd2016
	WHERE  secid = '108105'
	"""

	# Execute the query and fetch the data
	option_prices = db.raw_sql(query)
	option_prices = db.raw_sql(query)

	# Display the first few rows of the data
	print(option_prices.index)
	print(option_prices.head())

Loading library list...
Done
WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       393380, 393381, 393382, 393383, 393384, 393385, 393386, 393387, 393388,
       393389],
      dtype='int64', length=1893390)
      secid        date              symbol symbol_flag      exdate  \
0  108105.0  2016-01-04  SPX 160115C1000000           1  2016-01-15   
1  108105.0  2016-01-04  SPX 160115C1025000           1  2016-01-15   
2  108105.0  2016-01-04  SPX 160115C1050000           1  2016-01-15   
3  108105.0  2016-01-04  SPX 160115C1075000           1  2016-01-15   
4  108105.0  2016-01-04  SPX 160115C1100000           1  2016-01-15   

    last_date cp_flag  strike_price  best_bid  best_offer  ...  theta  \
0  2016-01-04       C     1000000.0    1007.4      1010.8  ...    NaN 

In [59]:
option_prices.index.is_unique

False

In [6]:
option_prices.columns

Index(['secid', 'date', 'symbol', 'symbol_flag', 'exdate', 'last_date',
       'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume',
       'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta',
       'optionid', 'cfadj', 'am_settlement', 'contract_size', 'ss_flag',
       'forward_price', 'expiry_indicator', 'root', 'suffix'],
      dtype='object')

In [22]:
option_prices.index

Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,
       ...
       393380, 393381, 393382, 393383, 393384, 393385, 393386, 393387, 393388,
       393389],
      dtype='int64', length=1893390)

In [24]:
option_prices.iloc[0]

secid                         108105.0
date                        2016-01-04
symbol              SPX 160115C1000000
symbol_flag                          1
exdate                      2016-01-15
last_date                   2016-01-04
cp_flag                              C
strike_price                 1000000.0
best_bid                        1007.4
best_offer                      1010.8
volume                          2000.0
open_interest                  42291.0
impl_volatility                    NaN
delta                              NaN
gamma                              NaN
vega                               NaN
theta                              NaN
optionid                   103574072.0
cfadj                              1.0
am_settlement                      1.0
contract_size                    100.0
ss_flag                              0
forward_price                     None
expiry_indicator                  None
root                              None
suffix                   

In [55]:
with wrds.Connection(wrds_username="mayurankv") as db:
	query = """
	SELECT secid FROM optionm_all.secnmd WHERE ticker = 'VIX'
	"""

	result_df = db.raw_sql(query)

	print(result_df)

Loading library list...
Done
      secid
0  117801.0
1  117801.0
2  117801.0
3  117801.0
4  117801.0
5  117801.0
6  117801.0
7  117801.0
8  117801.0


In [54]:
int()

108105

In [5]:
option_prices.to_csv("test.csv")

In [62]:
import pandas as pd


# option_prices = pd.read_csv("test.csv", index_col=0)

In [66]:
type(pd.IndexSlice[:])

slice

In [67]:
print(option_prices.head())

      secid        date              symbol  symbol_flag      exdate  \
0  108105.0  2016-01-04  SPX 160115C1000000            1  2016-01-15   
2  108105.0  2016-01-04  SPX 160115C1050000            1  2016-01-15   
3  108105.0  2016-01-04  SPX 160115C1075000            1  2016-01-15   
4  108105.0  2016-01-04  SPX 160115C1100000            1  2016-01-15   
5  108105.0  2016-01-04  SPX 160115C1125000            1  2016-01-15   

    last_date cp_flag  strike_price  best_bid  best_offer  ...  \
0  2016-01-04       C     1000000.0    1007.4      1010.8  ...   
2  2015-06-29       C     1050000.0     957.3       960.9  ...   
3  2015-06-29       C     1075000.0     932.3       935.9  ...   
4  2015-12-16       C     1100000.0     907.4       911.1  ...   
5  2015-08-28       C     1125000.0     882.3       886.1  ...   

   impl_volatility  delta  gamma  vega  theta     optionid  cfadj  \
0              NaN    NaN    NaN   NaN    NaN  103574072.0    1.0   
2              NaN    NaN    NaN

In [17]:
option_prices.iloc[0]

secid                         108105.0
date                        2016-01-04
symbol              SPX 160115C1000000
symbol_flag                          1
exdate                      2016-01-15
last_date                   2016-01-04
cp_flag                              C
strike_price                 1000000.0
best_bid                        1007.4
best_offer                      1010.8
volume                          2000.0
open_interest                  42291.0
impl_volatility                    NaN
delta                              NaN
gamma                              NaN
vega                               NaN
theta                              NaN
optionid                   103574072.0
cfadj                              1.0
am_settlement                      1.0
contract_size                    100.0
ss_flag                              0
forward_price                     None
expiry_indicator                  None
root                              None
suffix                   

In [31]:
option_prices[~option_prices["suffix"].isnull()]["suffix"]

Series([], Name: suffix, dtype: object)

In [34]:
option_prices.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [36]:
columns = option_prices.columns
unique_columns = option_prices.loc[:, (option_prices != option_prices.iloc[0]).any()].columns

In [35]:
unique_columns

Index(['date', 'symbol', 'exdate', 'last_date', 'cp_flag', 'strike_price',
       'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility',
       'delta', 'gamma', 'vega', 'theta', 'optionid', 'am_settlement',
       'forward_price', 'expiry_indicator', 'root', 'suffix'],
      dtype='object')

In [39]:
columns.difference(unique_columns)

Index(['cfadj', 'contract_size', 'secid', 'ss_flag', 'symbol_flag'], dtype='object')

In [26]:
option_prices[option_prices["suffix"].isnull()]["suffix"]

0         None
1         None
2         None
3         None
4         None
          ... 
393385    None
393386    None
393387    None
393388    None
393389    None
Name: suffix, Length: 1893390, dtype: object

In [19]:
option_prices.iloc[0]["impl_volatility"]

nan